In [1]:
install.packages("skimr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
library(dplyr)
library(skimr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
#reading in the raw data
car_data <- read.csv("raw_data/train.csv")
head(car_data)

,X,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5,,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7,,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5,,17.74
5,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5,,3.50
6,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8,21 Lakh,17.50


In [4]:
#size of the data
nrow(car_data)

[1] 5847

In [5]:
#getting rid of the units from the mileage column
car_data$Mileage <- sapply(car_data$Mileage, function(value) {
  if (grepl("kmpl", value)) {
    as.numeric(gsub("kmpl", "", value)) #removing the kmpl
  } else if (grepl("km/kg", value)) {
    as.numeric(gsub("km/kg", "", value)) * 1.40 #converitng km/kg to kmpl
  }else{
    NA
  }
})


In [6]:
#getting rid of the units in other column
car_data$Engine <- as.numeric(gsub("CC", "", car_data$Engine))
car_data$Power <- as.numeric(gsub("bhp", "", car_data$Power))
car_data$New_Price <- as.numeric(gsub("Lakh", "", car_data$New_Price))
head(car_data)

Warning message:
“NAs introduced by coercion”


,X,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5,NA,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199,88.70,5,8.61,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7,NA,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5,NA,17.74
5,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461,63.10,5,NA,3.50
6,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36,2755,171.50,8,21.00,17.50


In [7]:
#checking for the missing data
colSums(is.na(car_data) | car_data == "")

X              Name          Location              Year 
                0                 0                 0                 0 
Kilometers_Driven         Fuel_Type      Transmission        Owner_Type 
                0                 0                 0                 0 
          Mileage            Engine             Power             Seats 
                2                36                36                38 
        New_Price             Price 
             5049                 0

In [8]:
#mean and median of all the column
summary(car_data)

       X            Name             Location              Year     
 Min.   :   1   Length:5847        Length:5847        Min.   :1998  
 1st Qu.:1510   Class :character   Class :character   1st Qu.:2012  
 Median :3015   Mode  :character   Mode  :character   Median :2014  
 Mean   :3013                                         Mean   :2013  
 3rd Qu.:4518                                         3rd Qu.:2016  
 Max.   :6018                                         Max.   :2019  
                                                                    
 Kilometers_Driven  Fuel_Type         Transmission        Owner_Type       
 Min.   :    171   Length:5847        Length:5847        Length:5847       
 1st Qu.:  33468   Class :character   Class :character   Class :character  
 Median :  52576   Mode  :character   Mode  :character   Mode  :character  
 Mean   :  58410                                                           
 3rd Qu.:  72490                                                    

In [9]:
#finding the standerd deviation of all the columns
sapply(car_data, function(x) if(is.numeric(x)) sd(x, na.rm = TRUE) else NA)


X              Name          Location              Year 
     1.736399e+03                NA                NA      3.194949e+00 
Kilometers_Driven         Fuel_Type      Transmission        Owner_Type 
     9.237971e+04                NA                NA                NA 
          Mileage            Engine             Power             Seats 
     4.357132e+00      6.019726e+02      5.389672e+01      8.066681e-01 
        New_Price             Price 
     2.026768e+01      1.127597e+01

**1. Engine I choose median because the SD was pretty large which means our mean is skwed and replacing the missing value with the mean would skew it more to the right**
  **2. for others means was good replacement value since, the Standred error is not too big. I choose to drop the new_price column since, more than 90% of the values were missing.**

In [10]:
#replacing the missing data

car_data$Mileage[is.na(car_data$Mileage)] <- mean(car_data$Mileage,na.rm=TRUE)
car_data$Engine[is.na(car_data$Engine)] <- median(car_data$Engine,na.rm=TRUE)
car_data$Power[is.na(car_data$Power)] <- mean(car_data$Power,na.rm=TRUE)
car_data$Seats[is.na(car_data$Seats)] <- median(car_data$Seats,na.rm=TRUE)
car_data <- subset(car_data, select = -New_Price)

In [11]:
#making sure we got rid of the missing data
colSums(is.na(car_data) | car_data == "")

X              Name          Location              Year 
                0                 0                 0                 0 
Kilometers_Driven         Fuel_Type      Transmission        Owner_Type 
                0                 0                 0                 0 
          Mileage            Engine             Power             Seats 
                0                 0                 0                 0 
            Price 
                0

In [12]:
#one hot encoding on fuel_typle and transmission
encoded_df <- as.data.frame(model.matrix(~ Fuel_Type + Transmission - 1, data = car_data))
car_data_encoded <- cbind(car_data[, !(names(car_data) %in% c("Fuel_Type", "Transmission"))], encoded_df)
head(car_data_encoded)

,X,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_TypeDiesel,Fuel_TypeElectric,Fuel_TypePetrol,TransmissionManual
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582,126.20,5,12.50,1,0,0,1
2,2,Honda Jazz V,Chennai,2011,46000,First,18.20,1199,88.70,5,4.50,0,0,1,1
3,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248,88.76,7,6.00,1,0,0,1
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968,140.80,5,17.74,1,0,0,0
5,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461,63.10,5,3.50,1,0,0,1
6,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755,171.50,8,17.50,1,0,0,0


In [13]:
#adding a new column for the age of the car
mutated_car_data <- car_data %>%
  mutate(age = 2024 - Year)
head(mutated_car_data)

,X,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,age
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5,12.50,9
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199,88.70,5,4.50,13
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7,6.00,12
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5,17.74,11
5,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461,63.10,5,3.50,11
6,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36,2755,171.50,8,17.50,8


In [14]:
#renaming the new column that we created
mutated_car_data <- mutated_car_data %>%
  rename(car_age = age)
head(mutated_car_data)

,X,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,car_age
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5,12.50,9
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199,88.70,5,4.50,13
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7,6.00,12
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5,17.74,11
5,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461,63.10,5,3.50,11
6,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36,2755,171.50,8,17.50,8


In [15]:
#writing the new mutated_car_data dataframe to the file
if (!dir.exists("data_clean")) {
  dir.create("data_clean")
}
write.csv(mutated_car_data, "data_clean/clean_train.csv", row.names = FALSE)